In [2]:
# import packages
import json
import numpy as np

# Path of score_file & infringement/non-infringement files
scores_file = '/home/guangwei/LLM-COPYRIGHT/copyright_newVersion/scores/scores-char-copying.nonliteral.prompt1.Meta-Llama-3.1-8B.greedy.json'
infringement_output = '/home/guangwei/LLM-COPYRIGHT/copyright_newVersion/division_non/non.infringement.json'
non_infringement_output = '/home/guangwei/LLM-COPYRIGHT/copyright_newVersion/division_non/non.non_infringement.json'

# Process for lower/upper rouge_l 
def process_data(scores_file, infringement_output, non_infringement_output):
    with open(scores_file, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Split by ourselves
    scores = [entry['score_char_overlap'] for entry in data]

    lower_threshold = np.percentile(scores, 10)
    upper_threshold = np.percentile(scores, 90)
    
    print(f'Lower threshold (30th percentile): {lower_threshold}')
    print(f'Upper threshold (70th percentile): {upper_threshold}')

    group1 = [entry for entry in data if entry['score_char_overlap'] >= upper_threshold]
    group2 = [entry for entry in data if entry['score_char_overlap'] <= lower_threshold]

    for entry in group1:
        entry['label'] = 0

    for entry in group2:
        entry['label'] = 1
        
    group2 = group2[:len(group1)]  # Slice group2 to match the size of group1
    print(f'Group infringement: {len(group1)} entries')
    print(f'Group non-infringement: {len(group2)} entries')

    with open(infringement_output, 'w', encoding='utf-8') as file:
        json.dump(group1, file, ensure_ascii=False, indent=4)

    with open(non_infringement_output, 'w', encoding='utf-8') as file:
        json.dump(group2, file, ensure_ascii=False, indent=4)

    print(f'Data saved to {infringement_output} and {non_infringement_output}')

# Main function & Run
def main(scores_file, infringement_output, non_infringement_output):
    process_data(scores_file, infringement_output, non_infringement_output)

main(scores_file, infringement_output, non_infringement_output)

Lower threshold (30th percentile): 0.0
Upper threshold (70th percentile): 2.0
Group infringement: 68 entries
Group non-infringement: 68 entries
Data saved to /home/guangwei/LLM-COPYRIGHT/copyright_newVersion/division_non/non.infringement.json and /home/guangwei/LLM-COPYRIGHT/copyright_newVersion/division_non/non.non_infringement.json
